In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math as mt
import time
import sys

sys.path.insert(1, '/rds/general/user/lr4617/home/4th_Year_Project/Final_Year_Project/')

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE_
from scipy.spatial import distance
from scipy.stats import entropy
from mPE_ultis import integrand, ubble, array_list, permutation
from util import rolling_mean, probability, probability_v2, get_mPE_matrix
from sklearn.cluster import KMeans

In [2]:
rats = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/rats_sampling_10_window_150.npy')
lengths = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/lengths.npy')

to_be_removed = '.ipynb_checkpoints'

root_dir = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/'
normal_trajs = os.listdir(root_dir + 'normal')
drug_trajs_1 = os.listdir(root_dir + 'caffeine')
drug_trajs_2 = os.listdir(root_dir + 'amphetamine')

if to_be_removed in normal_trajs:
    idx = normal_trajs.index(to_be_removed)
    normal_trajs.pop(idx)
    
if to_be_removed in drug_trajs_1:
    idx = drug_trajs_1drug_trajs_1.index(to_be_removed)
    drug_trajs_1.pop(idx)

if to_be_removed in drug_trajs_2:
    idx = drug_trajs_2.index(to_be_removed)
    drug_trajs_2.pop(idx)

names = normal_trajs + drug_trajs_1 + drug_trajs_2

In [3]:
def mPE__(*argv):    
    g=len(argv)
    
    if g==2:
        n=argv[1]
        datain=argv[0]
        lamin=np.array([1])
        epsin=np.array([1])
    if g==3:
        n=argv[1]
        datain=argv[0]
        lamin=np.array([1])
        epsin=argv[2]
    if g==4:
        n=argv[1]
        datain=argv[0]
        lamin=argv[3]
        epsin=argv[2]
    
#    Check for the data to be in the right dimension        
    if datain.shape[0] == 1 or datain.shape[1] == 1:
        if len(datain[:,0])>len(datain[0,:]):
            datain=datain.transpose()
    else:
        if len(datain[:,1])>len(datain[1,:]):
            datain=datain.transpose()
        
    scalesno=len(epsin)
    lagno=len(lamin)
    HH=np.zeros((lagno,scalesno))
    norm_HH=np.zeros((lagno,scalesno))
    
#    Definition of parameters: fac is the number of possible permutations
#    Elem is the No of data points
#    Dim is the dimensionality of the samples
    
    for lam in range(0,lagno):
        for eps in range(0,scalesno):
            scale=epsin[eps]
            L=lamin[lam]
            xlen=len(datain[0,:])
            ylen=np.round((xlen/scale)-0.5)
            ylen=ylen.astype(int)
            D=np.zeros((len(datain[:,0]),ylen))
            for ylenc in range(0,ylen):
                dfg=datain[:,((ylenc)*scale):((ylenc+1)*scale)]
                fhk=np.sum(dfg,axis=1)
                r=(1/scale)*fhk
                D[:,ylenc]=r
                
            data=D
            fac=mt.factorial(n)
            elem=len(data[0,:])
            dim=len(data[:,0])
            
        #    A is a n-by-factorial(n) matrix that shows all the 
        #    possible permutations of n elements
            A=permutation(n)
            
        #    counter is a factorial(n) square matrix that counts the recurrence of
        #    a dim-dimensional permutation
            lpi=fac*np.ones((dim))
            lpi=lpi.astype(int)
            nj = lpi.tolist()
            counter=np.zeros((nj))
            
        #    For each iteration i, a series of n points is sampled from the 
        #    data set and the corresponding permutation is identified. 
        #    The counter matrix keeps track of the numiber of times a certain
        #    permutation, or combination of permutations is observed. 
            for i in range(0, elem-n*L+L-1):
                coord=np.zeros((dim))
                for num in range(0,dim):
                    sample=data[num,range(i,i+L*(n),L)]
                    ord=ubble(sample)
                    perm_num=0
                    check_1=1
                    check_2=1
                    
                    while (perm_num<=fac) and (check_2):
                       
                        check_1=1
                        for j in range(0,n-1):
                            if ord[j]!=A[perm_num,j]:
                                check_1=0
                        if check_1:
                            coord[num]=perm_num
                            check_2=0
                            
                        perm_num=perm_num+1
                
                    
                coord=coord.astype(int)
                coord1=tuple(coord)
                counter[coord1]=counter[coord1]+1
                
        #    Once the counter matrix is complete, each element is divided by the
        #    total number of samples to get a empirical probability, and the PE is
        #    computed according to the method described in Schurmann
            0
            H=0
            counter1=counter.flatten()
            for iter in range(0,(fac**dim)):
               
                rec=counter1[iter]
                tot=elem-n+1
                
                if rec==0:
                    H=H
                    
                else:
                    I = quad(integrand, 0, 1, args=(rec))
                    I=I[0]
                    coeff=((-1)**rec)*I
                    prob=(rec/tot)*(psi.digamma(tot)-psi.digamma(rec)-coeff)
                    H=H+mt.log2(mt.exp(prob))
                 
        #     The normalised entropy norm_H is computed as well
            norm_H=H/(mt.log2(fac**dim))
            
            HH[lam,eps]=H
            norm_HH[lam,eps]=norm_H
    return [HH,norm_HH]

In [9]:
################## PARAMS ##################
dims = np.arange(60)
span = 30
n_PC = 3
min_length = 334800
dims = np.arange(0, 60, 3)
############################################

mCPEs = np.zeros((12, 20))

for which_traj, length in enumerate(lengths):
    
    ##### retireving single trajectory #####
    if which_traj == 0:
        idx = 0
    else:
        idx += lengths[which_traj-1]

    traj = rats[idx:idx+length, :]
    
    ##### perform PCA #####
    pca = PCA(n_components=n_PC)
    reduced_traj = pca.fit_transform(traj)
    
    ##### make trajectories of standard lentgh #####
    reduced_traj = 300*reduced_traj[0:min_length, :]
    traj = 300*traj[0:min_length, :]
    
    print(traj.shape, reduced_traj.shape)
        
    ##### initiate velocity vector #####
    vel_reduced_signal = np.zeros((1, int(reduced_traj.shape[0]/span)))
    
    ##### populate velocity signal #####
    for iiii in range(1, reduced_traj.shape[0]-span, span):
        vel_reduced_signal[:, int(iiii/span)] = np.sum(np.asarray([distance.euclidean(reduced_traj[j-1, :] , reduced_traj[j, :]) for j in range(iiii, iiii+span)]))/span

    for dim in dims:

        print('dimension = ', int(dim/3))

        ##### retrieve marker signal (x,y,z) #####
        print(np.arange(dim, dim+3, 1))
        marker_signal = traj[:, dim:dim+3]

        ##### initiate velocity vector #####
        vel_signal = np.zeros((1, int(traj.shape[0]/span)))
        
        ##### populate velocity signal #####
        for iii in range(1, traj.shape[0]-span, span):
            vel_signal[:, int(iii/span)] = np.sum(np.asarray([distance.euclidean(marker_signal[j-1, :] , marker_signal[j, :]) for j in range(iii, iii+span)]))/span

        ##### calculate velocity signal mPE #####
        [mH, _] = mPE_(vel_signal, 3)

        ##### create joint signal for Joint Dynamical Entropy #####
        joint_signal = np.concatenate((vel_reduced_signal, vel_signal), axis=0)

        ##### caluclate joint entropy using joint signal ##### 
        [JH, _] = mPE_(joint_signal, 3)

        ##### calculate multdimensional Conditional Permutation Entropy #####
        mCPE = JH - mH
        
        mCPEs[which_traj, int(dim/3)] = mCPE
        
        print(mCPE)
            

(334800, 60) (334800, 3)
dimension =  0
[0 1 2]
[[2.15851894]]
dimension =  1
[3 4 5]
[[2.09210899]]
dimension =  2
[6 7 8]
[[2.11123639]]
dimension =  3
[ 9 10 11]
[[1.87595905]]
dimension =  4
[12 13 14]
[[1.97376888]]
dimension =  5
[15 16 17]
[[2.05764523]]
dimension =  6
[18 19 20]
[[1.97900213]]
dimension =  7
[21 22 23]
[[2.06220376]]
dimension =  8
[24 25 26]
[[2.11142852]]
dimension =  9
[27 28 29]
[[2.15700483]]
dimension =  10
[30 31 32]
[[2.01233801]]
dimension =  11
[33 34 35]
[[2.09261944]]
dimension =  12
[36 37 38]
[[1.92863887]]
dimension =  13
[39 40 41]
[[1.95485833]]
dimension =  14
[42 43 44]
[[2.07020467]]
dimension =  15
[45 46 47]
[[2.08486236]]
dimension =  16
[48 49 50]
[[2.16631141]]
dimension =  17
[51 52 53]
[[2.1992155]]
dimension =  18
[54 55 56]
[[2.21004973]]
dimension =  19
[57 58 59]
[[2.21618938]]
(334800, 60) (334800, 3)
dimension =  0
[0 1 2]
[[2.28477626]]
dimension =  1
[3 4 5]
[[2.26832253]]
dimension =  2
[6 7 8]
[[2.2994116]]
dimension =  3
[ 

In [ ]:
block_lengths = [sum(lengths[i*4:(i*4)+4]) for i in range(3)]
PCs = np.arange(1,5)
modes = ['normal', 'caffeine', 'amphetamine']
use_all = False
span = 30
n_PC = 3
max_length = 334800
dims = np.arange(0, 60, 3)

for i in range(len(block_lengths)):
    
    if i == 0:
        signal_mode = rats[0:block_lengths[0], :]
    else:
        signal_mode = rats[block_lengths[i-1]:block_lengths[i-1]+block_lengths[i], :]

    for ii in range(4):

        if ii == 0 :
            signal = signal_mode[0:lengths[(i*4)], :]
            if signal.shape[0] > max_length:
                signal = signal[0:max_length, :]
        else:
            signal = signal_mode[lengths[(i*4) + ii -1]:lengths[(i*4) + ii-1] + lengths[(i*4) + ii], :]
            if signal.shape[0] > max_length:
                signal = signal[0:max_length, :]

        pca = PCA(n_components=n_PC)
        reduced_traj = pca.fit_transform(signal)

        vel_signal = np.zeros((1, int(signal.shape[0]/span)))
        vel_reduced_signal = np.zeros((1, int(signal.shape[0]/span)))
    
        print(vel_signal.shape)
        print(vel_reduced_signal.shape)
        
        for iii in dims:
        
            for iiii in range(int(signal.shape[0]/span)):

                vel_signal[:, iiii] = np.sum(np.asarray([distance.euclidean(signal[j-1, iii+3] , signal[j, iii+3]) for for j in range(1, iiii+span)]))/span
                vel_reduced_signal[:, iiii] = np.sum(np.asarray([distance.euclidean(reduced_traj[j-1, :] , reduced_traj[j, :]) for for j in range(1, iiii+span)]))/span

            # calculate marker signal mPE
            [mH, _] = mPE__(vel_signal, 3)

            # calculate reduced signal mPE
            [rH, _] = mPE__(vel_reduced_signal, 3)

            # create joint signal for Joint Dynamical Entropy
            joint_signal = np.concatenate((vel_signal, vel_reduced_signal), axis=0)

            # caluclate joint entropy using joint signal
            [JH, _] = mPE__(joint_signal, 3)

            # calculate dynamical mutual information
            DMI = rH + mH - JH

            print('DMI = ', DMI)
            print('Reduced velocity PE = ', rH)
            print('Marker velocity PE = ', rH)
